In [1]:
!pip install tensorboard

In [2]:
import sys

import yaml
from net_architectures.lstm import *
from net_architectures.mlp import *
from gensim.models import Word2Vec
import pandas as pd
import random
#tensorboard --logdir=/home/diego/Documentos/MTG_drafter/draftsym/runs --verbosity 0

random.seed(42)

In [3]:
# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
fi = open('config.yaml', 'r')
card_dict = yaml.load(fi, Loader=yaml.FullLoader)

In [5]:
def indice_para_nome(indice, cards_dict):
    """Converte um índice para o nome da carta correspondente usando o dicionário de cartas."""
    try:
        return cards_dict['cards_list'][indice]
    except IndexError:
        raise ValueError(f"Índice {indice} fora do alcance do dicionário de cartas.")


In [6]:
df_train = pd.read_csv('dataset/train_df.csv')
df_train = df_train.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
#df_train = df_train[:90]

In [7]:
df_train.head(50)

,#_draft,#_round,pick,available,pool,label
0,1,1,Patient_Rebuilding,"['Divination', 'Arcane_Encyclopedia', 'Suspici...",[],235
1,1,2,Knightly_Valor,"['Druid_of_the_Cowl', 'Fountain_of_Renewal', '...",['Patient_Rebuilding'],217
2,1,3,Aerial_Engineer,"['Dwindle', 'Dwarven_Priest', 'Boggart_Brute',...","['Patient_Rebuilding', 'Knightly_Valor']",81
3,1,4,Marauder's_Axe,"['Skyrider_Patrol', 'Inspired_Charge', 'Goblin...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",243
4,1,5,Trusty_Packbeast,"['Aven_Wind_Mage', 'Dryad_Greenseeker', 'Draco...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",244
5,1,6,Trusty_Packbeast,"['Dwindle', 'Rustwing_Falcon', 'Doublecast', '...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",244
6,1,7,Omenspeaker,"['Aven_Wind_Mage', 'Bristling_Boar', 'Boggart_...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",256
7,1,8,Omenspeaker,"['Skyrider_Patrol', 'Bristling_Boar', 'Torment...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",256
8,1,9,Manalith,"['Omenspeaker', 'Centaur_Courser', 'Goblin_Mot...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",76
9,1,10,Gearsmith_Guardian,"['Totally_Lost', ""Ajani's_Welcome"", 'Root_Snar...","['Patient_Rebuilding', 'Knightly_Valor', 'Aeri...",248


: 

In [7]:
word_to_vec_model = Word2Vec.load("card2vec.model")

In [8]:
from dataset_class import *

# Crie o dataset e o DataLoader
dataset = MagicDataset(df_train, word_to_vec_model, card_dict)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Itere sobre as primeiras 45 entradas do DataLoader
for i, (inputs, labels, mascara) in enumerate(dataloader):
    if i < 45:
        print(f"--- Entrada {i+1} ---")
        
        # Converta os embeddings para nomes
        input_names = [MagicDataset.embedding_para_nome(embedding.numpy(), word_to_vec_model) for embedding in inputs.squeeze(0)]
        
        # Converta o índice da label para o nome usando a função indice_para_nome
        label_name = indice_para_nome(labels.item(), card_dict)
        print(labels.item())
        print("Inputs:", input_names)
        print("Label:", label_name)
        print("Máscara:", mascara)
        print("\n")
        break
    else:
        break


--- Entrada 1 ---
235
Inputs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label: Patient_Rebuilding
Máscara: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1

In [9]:
# Instanciar o modelo LSTM
from torch import nn
from net_architectures.mlp import *
#model = LSTMModel()
model = FeedForwardModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [11]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
from dataset_class import *
from torch.utils.data import DataLoader

# Defina os hiperparâmetros
num_epochs = 20
batch_size = 32
model_name = 'model_4'

# Configurar o TensorBoard SummaryWriter para escrever logs na pasta 'runs'
log_dir = './runs/'+model_name+'/'
writer = SummaryWriter(log_dir=log_dir)

# Defina um diretório para salvar os checkpoints
checkpoint_dir = 'checkpoints'

# Certifique-se de que o diretório exista (ou crie-o)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Criar o dataset e o DataLoader
dataset = MagicDataset(df_train, word_to_vec_model, card_dict)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Mover o modelo para a GPU
model.to(device)

# Ciclo de treinamento
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for i, (inputs, labels, mascara) in enumerate(dataloader):
        # Mover as entradas e rótulos para a GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        mascara = mascara.to(device)

        # Zerar os gradientes
        optimizer.zero_grad()

        # Forward
        outputs = model(inputs, mascara)
        loss = criterion(outputs, labels)
        
        # Acumular a perda
        total_loss += loss.item()

        # Backward
        loss.backward()
        optimizer.step()

        # Calcular acertos
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calcular e imprimir a perda média e a acurácia da época
    average_loss = total_loss / len(dataloader)
    epoch_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}, Epoch Acuracy: {epoch_accuracy:.4f}")

    # Registrar no TensorBoard
    writer.add_scalar('Epoch Accuracy', epoch_accuracy, epoch)
    writer.add_scalar('Epoch Loss', average_loss, epoch)

    # Salvar o estado do modelo a cada 5 épocas
    if (epoch + 1) % 5 == 0:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': average_loss
        }
        checkpoint_path = os.path.join(checkpoint_dir, f'{model_name}_checkpoint_epoch_{epoch + 1}.pt')
        torch.save(checkpoint, checkpoint_path)

# Fechar o writer após terminar o treinamento
writer.close()


Epoch [1/20], Average Loss: 1.3312, Epoch Acuracy: 0.5414
Epoch [2/20], Average Loss: 1.0875, Epoch Acuracy: 0.6028
Epoch [3/20], Average Loss: 1.0253, Epoch Acuracy: 0.6216
Epoch [4/20], Average Loss: 0.9978, Epoch Acuracy: 0.6303
Epoch [5/20], Average Loss: 0.9830, Epoch Acuracy: 0.6349
Epoch [6/20], Average Loss: 0.9738, Epoch Acuracy: 0.6377
Epoch [7/20], Average Loss: 0.9678, Epoch Acuracy: 0.6394
Epoch [8/20], Average Loss: 0.9636, Epoch Acuracy: 0.6404
Epoch [9/20], Average Loss: 0.9599, Epoch Acuracy: 0.6415
Epoch [10/20], Average Loss: 0.9570, Epoch Acuracy: 0.6425
Epoch [11/20], Average Loss: 0.9545, Epoch Acuracy: 0.6431
Epoch [12/20], Average Loss: 0.9524, Epoch Acuracy: 0.6436
Epoch [13/20], Average Loss: 0.9507, Epoch Acuracy: 0.6442
Epoch [14/20], Average Loss: 0.9487, Epoch Acuracy: 0.6446
Epoch [15/20], Average Loss: 0.9476, Epoch Acuracy: 0.6452
Epoch [16/20], Average Loss: 0.9457, Epoch Acuracy: 0.6456
Epoch [17/20], Average Loss: 0.9448, Epoch Acuracy: 0.6460
Epoch 